In [4]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

# Load the data (modify path if needed)
df = pd.read_csv("/Users/suryanshpatel/Projects/Directed Readings/Technical/src/data/Finaldata_combined.csv")

# Initialize MediaPipe Face Mesh with blendshape support
mp_face_mesh = mp.solutions.face_mesh
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)

# Container for all features
all_features = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    rgb_path = row['Full_Path_RGB']
    depth_path = row['Full_Path_Depth']
    label = row['Label']

    try:
        # Read RGB image
        image = cv2.imread(rgb_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Read Depth image (assumes grayscale float or 8bit)
        depth_image = cv2.imread(depth_path, cv2.IMREAD_GRAYSCALE)

        # Process with MediaPipe
        results = face_mesh.process(image_rgb)

        if results.multi_face_landmarks:
            landmarks = results.multi_face_landmarks[0]
            xyz = []
            for lm in landmarks.landmark:
                xyz.extend([lm.x, lm.y, lm.z])

            # Expression features placeholder (MediaPipe expression API can be added if needed)
            expression_features = []  # e.g., use dummy or separate logic if required

            # Depth features
            depth_mean = np.mean(depth_image)
            depth_std = np.std(depth_image)

            # Combine all features
            features = xyz + expression_features + [depth_mean, depth_std, label]
            all_features.append(features)
    except Exception as e:
        print(f"Skipping {rgb_path} due to error: {e}")
        continue

# Build column names dynamically
num_landmarks = 468
xyz_cols = [f"{axis}{i}" for i in range(num_landmarks) for axis in ['x', 'y', 'z']]
expression_cols = []  # Add if you use blendshapes
depth_cols = ['depth_mean', 'depth_std']
columns = xyz_cols + expression_cols + depth_cols + ['Label']

# Create DataFrame
features_df = pd.DataFrame(all_features, columns=columns)

# Save to CSV for training
features_df.to_csv("pain_features.csv", index=False)
print("Saved feature DataFrame with shape:", features_df.shape)


I0000 00:00:1744060618.441747  777683 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M2
W0000 00:00:1744060618.444579  785212 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
100%|██████████| 54385/54385 [09:50<00:00, 92.14it/s] 


Saved feature DataFrame with shape: (7898, 1407)


In [6]:
features_df.shape

(7898, 1407)

I0000 00:00:1744233100.754527  777683 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M2


RuntimeError: Unable to open file at /Users/suryanshpatel/Projects/Directed Readings/Technical/src/face_landmarker.task

In [14]:
df.shape

(54385, 7)

In [13]:
features_df.shape


(7898, 1407)

In [11]:
features_df.head()

,x0,y0,z0,x1,y1,z1,x2,y2,z2,x3,...,z465,x466,y466,z466,x467,y467,z467,depth_mean,depth_std,Label
0,0.534029,0.450986,-0.003032,0.537100,0.426559,-0.014693,0.534876,0.433376,-0.006165,0.532415,...,-0.007273,0.557662,0.334991,-0.000862,0.559195,0.326888,-0.000831,4.469501,1.092254,0
1,0.535109,0.462379,-0.005626,0.532303,0.433103,-0.018389,0.535053,0.439488,-0.008331,0.527432,...,-0.010039,0.563861,0.330486,-0.010843,0.566077,0.321636,-0.011568,4.469975,1.094820,0
2,0.523983,0.470786,-0.002771,0.519819,0.449220,-0.016546,0.522267,0.451690,-0.006387,0.515115,...,-0.011112,0.551678,0.341565,-0.010308,0.553765,0.332091,-0.010935,4.470196,1.095579,0
3,0.526209,0.460931,-0.001444,0.522617,0.443690,-0.014566,0.525091,0.445433,-0.005214,0.519022,...,-0.009853,0.553484,0.346150,-0.008494,0.555271,0.338751,-0.009093,4.471546,1.089395,0
4,0.532931,0.464505,-0.002699,0.532458,0.441020,-0.017106,0.532687,0.445444,-0.006506,0.526321,...,-0.011079,0.558830,0.333272,-0.007666,0.560583,0.323697,-0.008000,4.469280,1.100579,0


In [12]:
# All XYZ columns
landmark_cols = [col for col in features_df.columns if col.startswith(('x', 'y', 'z'))]

# Get landmark data for the first face
landmarks_face_0 = features_df.loc[0, landmark_cols].values.reshape(468, 3)

# landmarks_face_0 is a numpy array of shape (468, 3)
